In [12]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
df_train = pd.read_csv('./data/preprocessed_DNN_train.csv', low_memory=False)

In [4]:
df_train.Attack_label.value_counts()

0    1091198
1     436538
Name: Attack_label, dtype: int64

In [5]:
df_train.Attack_type.value_counts()

0     1091198
4       97253
5       54351
9       40661
3       40050
2       40021
1       39946
6       38835
7       29446
10      19221
11      15982
8       12053
12       7751
13        682
14        286
Name: Attack_type, dtype: int64

In [7]:
# ssmote
smote = SMOTE()

In [8]:
# 二分类
_, df_binary = train_test_split(df_train, test_size=0.2, stratify=df_train.Attack_label)
x = df_binary.drop(['Attack_label', 'Attack_type'], axis=1)
y = df_binary['Attack_label']
x_smote, y_smote = smote.fit_resample(x, y)
train_balance_binary = pd.concat([x_smote, y_smote], axis=1)

In [9]:
train_balance_binary.Attack_label.value_counts()

1    218240
0    218240
Name: Attack_label, dtype: int64

In [11]:
train_balance_binary.to_csv('./data/preprocessed_DNN_train_balance_binary.csv', index=False, encoding='utf-8')

In [17]:
# 多分类
# 下采样到
under_sample_labels = [0, 4, 5]
over_sample_labels = [1, 2, 3, 6, 7, 8, 9, 10, 11, 12, 13, 14]
train_balance_multis = []
df = df_train.drop(['Attack_label'], axis=1)
for label in tqdm(under_sample_labels):
    label_df_train = df.loc[df.Attack_type == label, :].sample(50000)
    train_balance_multis.append(label_df_train)

for label in tqdm(over_sample_labels):
    label_df_train = df.loc[df.Attack_type == label, :]
    target_df_train = train_balance_multis[0]
    to_balance = pd.concat([label_df_train, target_df_train])
    x = to_balance.drop(['Attack_type'], axis=1)
    y = to_balance['Attack_type']
    x_smote, y_smote = smote.fit_resample(x, y)
    balance = pd.concat([x_smote, y_smote], axis=1)
    label_df_train = balance.loc[balance.Attack_type == label,:]
    train_balance_multis.append(label_df_train)

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [02:41<00:00, 13.47s/it]


In [18]:
train_balance_multi = pd.concat(train_balance_multis)

In [19]:
train_balance_multi.Attack_type.value_counts()

14    50000
13    50000
12    50000
11    50000
10    50000
9     50000
8     50000
7     50000
6     50000
5     50000
4     50000
3     50000
2     50000
1     50000
0     50000
Name: Attack_type, dtype: int64

In [22]:
train_balance_multi.to_csv('./data/preprocessed_DNN_train_balance_multi.csv', index=False, encoding='utf-8')

In [23]:
serve_df = pd.read_csv('data/preprocessed_DNN_test.csv', low_memory=False)